In [ ]:
import json
import os
import pandas as pd

from util.config import versioned

matched_all = versioned.read_parquet("matched.parquet")
matched_all = matched_all[matched_all["overall_score"] >= 11]
matched_all

In [ ]:
people_krs = pd.DataFrame(versioned.read_jsonl("people_krs.jsonl"))
people_krs

In [ ]:
people_pkw = pd.DataFrame(versioned.read_jsonl("people_pkw.jsonl"))
people_pkw

In [ ]:
employment_ends = pd.DataFrame({'employed_end': matched_all["employed_end"]})
employment_ends

In [ ]:
import matplotlib.pyplot as plt

# 2. Convert the 'event_date' column to datetime objects
df['employed_end'] = pd.to_datetime(df['employed_end'])

# 3. Extract the year and count the occurrences
yearly_counts = df['employed_end'].dt.year.value_counts()

# 4. Sort the counts by year (the index) for a chronological chart
yearly_counts = yearly_counts.sort_index()

# 5. Create the bar chart
plt.figure(figsize=(8, 5))
ax = yearly_counts.plot(
    kind='bar',
    color='coral',
    title='Number of Events per Year'
)

ax.set_xlabel("Year")
ax.set_ylabel("Number of Occurrences")
plt.xticks(rotation=0) # Keep year labels horizontal
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 3. Extract the year and count the occurrences
company_counts = people_krs['employed_krs'].value_counts()

# 4. Sort the counts by year (the index) for a chronological chart
company_counts = company_counts.sort_values()

# 5. Create the bar chart
plt.figure(figsize=(8, 5))
ax = company_counts.plot(
    kind='bar',
    color='coral',
    title='Number of Events per Year'
)

ax.set_xlabel("Year")
ax.set_ylabel("Number of Occurrences")
plt.xticks(rotation=0) # Keep year labels horizontal
plt.tight_layout()
plt.show()

company_counts